---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

### Housing Prices Dataset
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.

In [3]:



get_house_prices()

1996-04   1996-05   1996-06   1996-07   1996-08  \
State   RegionName                                                      
Alabama Adamsville    60200.0   60800.0   61000.0   60500.0   60200.0   
        Alabaster    108400.0  108300.0  107800.0  107500.0  107300.0   
        Albertville   67500.0   67200.0   66900.0   66200.0   65600.0   
        Arab          75000.0   74600.0   74400.0   73900.0   73500.0   
        Ardmore           NaN       NaN       NaN       NaN       NaN   
...                       ...       ...       ...       ...       ...   
Wyoming Burns             NaN       NaN       NaN       NaN       NaN   
        Casper            NaN       NaN       NaN       NaN       NaN   
        Cheyenne          NaN       NaN       NaN       NaN       NaN   
        Evansville        NaN       NaN       NaN       NaN       NaN   
        Pine Bluffs       NaN       NaN       NaN       NaN       NaN   

                      1996-09   1996-10   1996-11   1996-12   1997-01  ...  \
State   RegionName                                                     ...   
Alabama Adamsville    60400.0   60600.0   60400.0   60600.0   61100.0  ...   
        Alabaster    107000.0  107200.0  107600.0  108000.0  108300.0  ...   
        Albertville   65100.0   64800.0   64600.0   64700.0   64800.0  ...   
        Arab          73800.0   74300.0   74700.0   75100.0   75400.0  ...   
        Ardmore           NaN       NaN       NaN       NaN       NaN  ...   
...                       ...       ...       ...       ...       ...  ...   
Wyoming Burns             NaN       NaN       NaN       NaN       NaN  ...   
        Casper            NaN       NaN       NaN       NaN       NaN  ...   
        Cheyenne          NaN       NaN       NaN       NaN       NaN  ...   
        Evansville        NaN       NaN       NaN       NaN       NaN  ...   
        Pine Bluffs       NaN       NaN       NaN       NaN       NaN  ...   

                     2015-11  2015-12  2016-01  2016-02  2016-03  2016-04  \
State   RegionName                                                          
Alabama Adamsville     73900    73500    73600    74100    74800    75200   
        Alabaster     151800   152400   152800   153400   154200   154900   
        Albertville    91600    91500    91400    92000    92600    92600   
        Arab          112500   112100   111000   109800   109300   109500   
        Ardmore       144200   143100   142200   142500   143000   143200   
...                      ...      ...      ...      ...      ...      ...   
Wyoming Burns         164100   164500   166800   168500   169500   171100   
        Casper        182300   182600   184000   184500   184500   184900   
        Cheyenne      190500   191900   193800   194700   194800   195500   
        Evansville    303900   306600   308200   309700   310400   310200   
        Pine Bluffs   169600   169300   168400   166300   165800   166600   

                     2016-05  2016-06  2016-07  2016-08  
State   RegionName                                       
Alabama Adamsville     74900    74700    74700    74700  
        Alabaster     155200   155200   155600   156100  
        Albertville    92400    92400    92300    92100  
        Arab          110100   110700   111600   112400  
        Ardmore       143400   143100   142300   141600  
...                      ...      ...      ...      ...  
Wyoming Burns         172100   171600   170700   170300  
        Casper        185400   185200   184600   184100  
        Cheyenne      196400   197600   198800   199400  
        Evansville    308800   308600   308800   309300  
        Pine Bluffs   164500   160300   158000   157300  

[10730 rows x 245 columns]

### GDB Dataset
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

In [4]:
    def get_GDP():
        ## GDP Dataset
        GDP=pd.read_excel('gdplev.xls',header=5,usecols='E:G')
        GDP=GDP.rename(columns={str(GDP.columns[0]):'Quarter'}).set_index('Quarter')
        GDP=GDP.iloc[GDP.index.get_loc('2000q1'):]   
        GDP.columns=GDP.columns.map(lambda c: re.sub(r'[^a-zA-Z\s209]+',"",c))
        return GDP

In [5]:
get_GDP()

,GDP in billions of current dollars,GDP in billions of chained 2009 dollars
Quarter,,
2000q1,10031.0,12359.1
2000q2,10278.3,12592.5
2000q3,10357.4,12607.7
2000q4,10472.3,12679.3
2001q1,10508.1,12643.3
...,...,...
2015q2,17998.3,16374.2
2015q3,18141.9,16454.9
2015q4,18222.8,16490.7


## Solutions

In [6]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # State names Extraction
    U_States=pd.read_csv('university_towns.txt.',sep='\n',header=None)[0]
    states_drop=U_States[U_States.str.contains('edit')] # States extraction: 'State[edit]'
    states=states_drop.map(lambda s: s[:-6])            # drop '[edit]'
    states.name='State'
    States=pd.Series(np.zeros(U_States.size),name="State").map(lambda x: np.nan)
    States[states.index]=states
    States.ffill(inplace=True)
    #print(States)

    # Town Name Extraction
    U_Towns=pd.read_csv('university_towns.txt.',sep='\n',header=None).drop(states_drop.index)[0].str.split(' \(').map(lambda r: r[0])
    U_Towns.name='RegionName'
    #print(U_Towns)

    # University name Extraction
    U_name=pd.read_csv('university_towns.txt.',sep='\n',header=None).drop(states_drop.index)[0].str.split(' \(').map(lambda r: r[-1].split(')')[0])
    U_name.name='University'
    #print(U_name)

    # University Towns DataFrame
    U_df=pd.DataFrame([States,U_Towns,U_name]).T.drop(states_drop.index).set_index(['State','RegionName'])
    return U_df

In [7]:
get_list_of_university_towns().reset_index()#[get_list_of_university_towns().duplicated()]

,State,RegionName,University
0,Alabama,Auburn,Auburn University
1,Alabama,Florence,University of North Alabama
2,Alabama,Jacksonville,Jacksonville State University
3,Alabama,Livingston,University of West Alabama
4,Alabama,Montevallo,University of Montevallo
...,...,...,...
512,Wisconsin,River Falls,University of Wisconsin–River Falls
513,Wisconsin,Stevens Point,University of Wisconsin–Stevens Point
514,Wisconsin,Waukesha,Carroll University
515,Wisconsin,Whitewater,University of Wisconsin–Whitewater


* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.

In [8]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    df=get_GDP().drop(get_GDP().columns[0],axis=1)
    rec=[]
    index=0
    while index < (len(df)-2):
        q0=df.iloc[index].values[0]
        q1=df.iloc[index+1].values[0]
        q2=df.iloc[index+2].values[0]
    ## Para determinar el inicio de una recesión solo se compara el ciclo actual y los dos siguientes:    
        if(q1<q0 and q2<q1):
            rec.append(index)
        index+=3
        
    return df.iloc[rec].iloc[0].name

In [9]:
#get_recession_start()

In [60]:
def get_recession_end():
    df=get_GDP().drop(get_GDP().columns[0],axis=1)
    index=df.index.get_loc(get_recession_start())
    rec=[]
    while index < (len(df)-2):
        q0=df.iloc[index].values[0]
        q1=df.iloc[index+1].values[0]
        q2=df.iloc[index+2].values[0]
        if(q1>q0 and q2>q1): 
            rec.append(index)
        index+=3
        
    return df.iloc[rec].iloc[0].name

In [62]:
#get_recession_end()

In [12]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    df=get_GDP().drop(get_GDP().columns[0],axis=1)
    bottom_index=df.index.get_loc(get_recession_end())-2
    
    return df.iloc[bottom_index].name

In [23]:
get_recession_bottom()

'2009q2'

* A quarter is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.

In [14]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df=pd.DataFrame.copy(get_house_prices())
    start_quarter=df.columns.get_loc('2000-01')
    df_quarter=df.iloc[:,start_quarter:].groupby(np.arange( len(df.iloc[:,start_quarter:].columns) )//3,axis=1).mean()
    
    init_year=int(df.iloc[:,start_quarter].name[:4])
    last_year=int(df.iloc[:,-1].name[:4]) 
    
    df_quarter.columns= [ str(y)+q for y in range(init_year,last_year+1) for q in ['q1','q2','q3','q4'] ] [:-1] #do not return 2016q4
    return df_quarter

In [15]:
convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Casper        89233.333333   89600.000000   89733.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Casper        93166.666667   95500.000000   97633.333333   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Casper        99433.333333  100633.333333  101733.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   

                            2002q2  ...         2014q2         2014q3  \
State   RegionName                  ...                                 
Alabama Adamsville    73133.333333  ...   77066.666667   75966.666667   
        Alabaster    128000.000000  ...  147133.333333  147633.333333   
        Albertville   76366.666667  ...   84033.333333   84766.666667   
        Arab          87700.000000  ...  113366.666667  111700.000000   
        Ardmore                NaN  ...  140533.333333  139566.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Casper       101533.333333  ...  175766.666667  177300.000000   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Adamsville    71900.000000   71666.666667   73033.333333   
        Alabaster    148700.000000  148900.000000  149566.666667   
        Albertville   86800.000000   88466.666667   89500.000000   
        Arab         111600.000000  110166.666667  109433.333333   
        Ardmore

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

In [58]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Recession Period indexes
    start = convert_housing_data_to_quarters().columns.get_loc(get_recession_start())
    bottom = convert_housing_data_to_quarters().columns.get_loc(get_recession_bottom())
    
    # House Prices by Quarters DF (only recession period+prior quarter)
    house_recession_quarters = convert_housing_data_to_quarters().iloc[:,start-1:bottom+1]
    
    # Non University Towns Houses Prices (only recession period+prior quarter)
    non_U_Towns = house_recession_quarters.drop(get_list_of_university_towns().index.get_level_values(level=1).tolist(),level=1)
    
    # University Towns Houses Prices (only recession period)
    U_Towns = get_list_of_university_towns().merge( house_recession_quarters ,how='inner',left_index=True,right_index=True).drop(['University'],axis=1)

    # T-Test (p<0.01 signifcance level)
        # Alternative hypothesis: non-U and U Towns are the same (both affected by recession)
        # Null hypothesis: non-U and U Towns are the differente (U Towns less affected by recession)
        
    price_ratio_U = U_Towns.iloc[:,0]/U_Towns.iloc[:,bottom-start+1]
    price_ratio_non_U = non_U_Towns.iloc[:,0]/non_U_Towns.iloc[:,bottom-start+1] 
    [t,p]=ttest_ind(price_ratio_U,price_ratio_non_U,nan_policy='omit')
    return (p<0.01,p,'university town' if price_ratio_U.mean()<price_ratio_non_U.mean() else 'non-university town')

In [59]:
run_ttest()#.iloc[517]#.dropna()

(True, 0.002896770432275549, 'non-university town')

In [51]:
(2,True if (2>1) else False)

(2, True)